In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import gradio as gr

In [ ]:
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",  # Add your API key here
        model_name="llama-3.3-70b-versatile"
    )
    return llm

In [ ]:
def create_vector_db():
    loader = DirectoryLoader("./data", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

In [ ]:
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """
You are a compassionate mental health support assistant designed to:
1. Analyze user sentiment (sadness, anxiety, stress, happiness).
2. Provide emotional support and coping strategies.
3. Escalate to professionals if crisis detected.

**Strict Rules**:
- NEVER diagnose or give medical advice.
- If user mentions self-harm/suicidal thoughts, respond:
  "I’m deeply concerned. Please contact [crisis hotline] immediately: [INSERT_LOCAL_HOTLINE]."

**Context** (for knowledge grounding):
{context}

**User Input**:
{question}

**Response Guidelines**:
1. **Start empathetically**: "I hear you. How can I support you today?"
2. **Validate**: "That sounds really difficult. You’re not alone."
3. **Ask open-ended questions**: "What’s been on your mind lately?"
4. **Suggest coping strategies** (tailored to sentiment):
   - Anxiety: "Try box breathing: inhale 4s, hold 4s, exhale 4s."
   - Sadness: "Small acts of self-care can help. What usually comforts you?"
5. **Close supportively**: "I’m here whenever you need to talk."

**Final Answer** (combine context + guidelines):
"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

In [ ]:


print("Initializing Chatbot.........")
llm = initialize_llm()

db_path = "./chroma_db"

if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(message, history):
    if not message.strip():
        return "Please provide a valid input"
    response = qa_chain.run(message)
    return response

with gr.Blocks(theme=gr.themes.Soft(
    primary_hue="teal",
    secondary_hue="emerald",
    neutral_hue="slate",
    text_size="lg",
    spacing_size="lg",
    radius_size="lg",
)) as app:
    
    gr.Markdown("""
    <div style='text-align: center; background: linear-gradient(to right, #0f766e, #0d9488); 
    padding: 20px; border-radius: 10px; color: white;'>
    <h1 style='margin-bottom: 10px;'>🧠 MindCare Companion 🤖</h1>
    <p style='margin-top: 0;'>Your compassionate mental health support chatbot</p>
    </div>
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("""
            ### About This Chatbot
            - Provides general mental health support
            - Offers non-judgmental listening
            - Suggests coping strategies
            """)
            
            gr.HTML("""
            <div style='background: #f0fdfa; padding: 15px; border-radius: 10px; border-left: 4px solid #0d9488;'>
            <p><strong>Note:</strong> For emergencies, please contact a licensed professional or call your local helpline.</p>
            </div>
            """)
            
        with gr.Column(scale=2):
            chatbot = gr.ChatInterface(
                fn=chatbot_response,
                title="",
                examples=[
                    "How can I manage anxiety?",
                    "I'm feeling depressed today",
                    "Suggest some stress relief techniques"
                ],
                submit_btn="Send",
                retry_btn="Regenerate",
                undo_btn="Delete",
                clear_btn="Clear",
            )
    
    gr.Markdown("""
    <div style='text-align: center; margin-top: 20px; color: #64748b; font-size: 0.9em;'>
    Confidential and secure | Your privacy is protected
    </div>
    """)

app.launch(
    height=800,
    favicon_path="https://emojicdn.elk.sh/🧠",  # Optional brain emoji favicon
)

e:\anaconda3\envs\medi_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing Chatbot.........


C:\Users\HP\AppData\Local\Temp\ipykernel_10144\3844665402.py:83: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\HP\AppData\Local\Temp\ipykernel_10144\3844665402.py:84: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embed

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [2]:
app.close()

Closing server running on port: 7861
